In [1]:
import tweepy
from tweepy import API
from kafka import KafkaConsumer, KafkaProducer
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json

### Twitter API access

In [2]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

# Kafka Producer

In [3]:
producer = KafkaProducer(bootstrap_servers='localhost:9092',api_version=(0,1,0))
topic_name = 'bots'

In [4]:
def get_twitter_data():
    hashtags = ['#RiyazNaikoo OR #CleanKashmir','#Aurangabad','#CongressForIndia OR #ArrestArnab',\
                '#NarendraModi OR #NaMo OR #BJPIndia OR #bjpindia OR #AmitShah OR #ArvindKejriwal OR #BJP',\
                '#HumModiKeSathHain OR #WhyBJPhatesDelhi','#IndianPolitics OR #IForIndia',\
                '#RahulFightsForIndia OR #UddhavMustAnswers','#isupportSudhirchaudhary OR #ISupportSudhirChoudhary',\
                '#IndiaWithZee OR #IndiaIsNotWithZeeNews']
    for hashtag in hashtags:
        res = api.search(hashtag,tweet_mode='extended')

        for tweet in res:
            if not tweet.user.verified:
                screen_name = tweet.user.screen_name
                screen_name = screen_name.replace('\n',' ').replace('\r',' ')
                description = tweet.user.description
                description = description.replace('\n',' ').replace('\r',' ')
                location = tweet.user.location
                location=location.replace('\n',' ')
                followers_count = tweet.user.followers_count
                friends_count = tweet.user.friends_count            
                listedcount = tweet.user.listed_count
                favourites_count = tweet.user.favourites_count
                statuses_count = tweet.user.statuses_count
                default_profile = tweet.user.default_profile

                row = {
                    "screen_name" : str(screen_name),
                    "description" : str(description),
                    "location" : str(location),
                    "followers_count": followers_count,
                    "friends_count" : friends_count,
                    "listedcount" : listedcount,
                    "favourites_count": favourites_count,
                    "statuses_count" : statuses_count,
                    "default_profile" : default_profile
                }

                print(row)

                producer.send(topic_name, value = json.dumps(row).encode('utf-8'))

In [3]:
def periodic_work(interval):
    while True:
        get_twitter_data()

### Kafka Data Streaming

In [ ]:
periodic_work(1)

In [ ]:
producer.flush()

In [ ]:
producer.close()